## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

In [2]:
!pip install gym==0.7.4

  Created wheel for gym: filename=gym-0.7.4-py3-none-any.whl size=204690 sha256=7de46834d9551ffbaedfe23d7aa3e32ff6f8fb206c1ac213124b7d3ea8bb2121
  Stored in directory: c:\users\91984\appdata\local\pip\cache\wheels\28\9f\e8\5acdfe381cbae5bac0aafe5cf17603c6d1386952b2967331dd
Successfully built gym


In [1]:
%run TCGame_Env1.py

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [2]:
from TCGame_Env1 import TicTacToe
import collections
import numpy as np
import random
import pickle
import time
from matplotlib import pyplot as plt

In [3]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [4]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [5]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [6]:
# Defining epsilon-greedy policy. You can choose any function epsilon-decay strategy
import random
def epsilon_greedy(state, time):
    epsilon = - 1/ (1 + np.exp((-time+7500000)/1700000)) + 1
    z = np.random.random()
        
    if z > epsilon:
        state1 = Q_state(state)
        action = max(Q_dict[state1],key=Q_dict[state1].get)
    else:
        agent_actions, env_actions = env.action_space(state)
        
        action = random.choice(list(agent_actions))
    
    return action




#### Tracking the state-action pairs for checking convergence - write your code here

In [7]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)
Q_dict = collections.defaultdict(dict)

States_track = collections.defaultdict(dict)



print(len(Q_dict))
print(len(States_track))


0
0


In [8]:
# Initialise states to be tracked
def initialise_tracking_states():
    sample_q_values = [('x-x-x-x-x-x-x-x-x',(6,5)),('x-x-x-x-x-x-x-x-x',(1,9)),
                       ('x-3-x-x-1-x-x-x-x',(7,5)),('x-5-x-x-x-x-5-7-x',(8,2))]
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []    #this is an array which will have appended values of that state-action pair for every 2000th episode         




In [9]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [10]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action])

In [11]:
initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [12]:
#Defining parameters for the experiment

#EPISODES = 20
EPISODES = 10000
#STEPS = 2
LR = 0.01                   #learning rate
GAMMA = 0.91


threshold = 2000
policy_threshold = 30000

### Q-update loop ---write your code here

In [13]:
start_time = time.time()

for episode in range(EPISODES):
    ##### Start writing your code from the next line
    env = TicTacToe()
    curr_state = env.state
    isTerm = False
    add_to_dict(curr_state)
    tot_reward = 0

    while not isTerm:
        current_state_ele = Q_state(curr_state)
        curr_action = epsilon_greedy(curr_state, episode)
        next_state, reward, isTerm = env.step(curr_state, curr_action)

        next_state_ele = Q_state(next_state)
        add_to_dict(next_state)

        if isTerm:
            Q_dict[current_state_ele][curr_action] += LR * (
                (reward - Q_dict[current_state_ele][curr_action]))
        else:
            max_next = max(Q_dict[next_state_ele],
                           key=Q_dict[next_state_ele].get)
            Q_dict[current_state_ele][curr_action] += LR * (
                (reward + (GAMMA * (Q_dict[next_state_ele][max_next]))) -
                Q_dict[current_state_ele][curr_action])

        curr_state = next_state
        tot_reward += reward

    # Tracking the Q-Values here
    
    if (episode == threshold-1):        #at the 1999th episode
        initialise_tracking_states()

    if ((episode+1) % threshold) == 0:   #every 2000th episode
        save_tracking_states()
        save_obj(States_track,'States_tracked')

    # Saving the Policy here
    
    if ((episode+1)% policy_threshold ) == 0:  #every 30000th episodes, the Q-dict will be saved
        save_obj(Q_dict,'Policy')
    
elapsed_time = time.time() - start_time
save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')

#### Check the Q-dictionary

In [14]:
Q_dict

defaultdict(dict,
            {'x-x-x-x-x-x-x-x-x': {(0, 1): -0.8823250088384367,
              (0, 3): -0.8764086669816782,
              (0, 5): -0.8819709809948397,
              (0, 7): -2.269716800885056,
              (0, 9): -2.194426711239846,
              (1, 1): -0.8759395228477387,
              (1, 3): -0.8708836382362931,
              (1, 5): -0.9008561499471037,
              (1, 7): -2.029855403308474,
              (1, 9): -2.0933365220081024,
              (2, 1): -0.8771931775358538,
              (2, 3): -0.8869622997268305,
              (2, 5): -0.8769415246442479,
              (2, 7): -1.951250345015059,
              (2, 9): -1.8066600120306409,
              (3, 1): -0.8675252381473333,
              (3, 3): -0.884418207629478,
              (3, 5): -0.8612586901112178,
              (3, 7): -1.9066166925775843,
              (3, 9): -2.0906392876896387,
              (4, 1): -0.8795023287046472,
              (4, 3): -0.8848119833888857,
              (4, 5)

In [15]:
len(Q_dict)

24765

In [16]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated
States_track

defaultdict(dict,
            {'x-x-x-x-x-x-x-x-x': {(6, 5): [-0.3638145139361288,
               -0.5806649593663216,
               -0.6913318421784381,
               -0.8009538214290988,
               -0.8638789593393594],
              (1, 9): [-0.6340869641384635,
               -1.4831387110485796,
               -1.4866230901425928,
               -1.5609777201778923,
               -2.0933365220081024]},
             'x-3-x-x-1-x-x-x-x': {(7, 5): []},
             'x-5-x-x-x-x-5-7-x': {(8, 2): []}})

#### Check the states tracked for Q-values convergence
(non-evaluative)

In [0]:
# Write the code for plotting the graphs for state-action pairs tracked

### Epsilon - decay check

In [17]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,5000000)
epsilon = []
for i in range(0,5000000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i))

In [18]:
plt.plot(time, epsilon)
plt.show()